In [70]:
def createFolder(name):
    if not os.path.exists(name):
        os.makedirs(name)
        

In [87]:
ticker = time.time()
folderName ="medlinks"
createFolder(folderName)
import time, requests, codecs ,json , re,time
from bs4 import BeautifulSoup
from lxml import html
import pandas
s = requests.Session()

url = 'http://www.medlinks.ru/modules.php?op=modload&name=Forum&file=index'

page =  s.get(url)
soup = BeautifulSoup(page.content, 'lxml')

mu = soup.find('body')
mu = mu.find("div")
mu = mu.find_all("table")

mu = mu[28].find_all("tr")[2:]

links = []
for i in range(0,len(mu)):
    tds = mu[i].find_all('td')
    if(len(tds) > 2):
        a = tds[1].find('a')['href']
        links.append(a)

links = links[:-6]      
links = ['http://www.medlinks.ru/' + i for i in links]

with open('topic-urls.txt', 'w+') as f:
    for i in links:
        f.write(i + '\n')
num = 31
urls =[]
pageNum = 0
errorNum =0
for i in links[31:]:
    print(i)
    print('-----')
    page =  s.get(i)
    soup = BeautifulSoup(page.content, 'lxml')
    mu = soup.find('body')
    mu = mu.find("div")
    mu = mu.find_all("table")
    mu = mu[28].find_all("tr")[1:]
    d={}
    if mu[len(mu)-2].find_all("a"):
        mids=mu[len(mu)-2].find_all("a")[-1] 
        temp = mids["href"].split("=")[-1]
        temp = int(temp)
    else:
        temp = 0
    print(temp)
        
    while temp >= 0:
        page =  s.get(i + "&start="+ str(temp))
        soup = BeautifulSoup(page.content, 'lxml')
        mu = soup.find('body')
        mu = mu.find("div")
        mu = mu.find_all("table")
        mu = mu[28].find_all("tr")[1:]

        for j in range(0,len(mu)):
            tds = mu[j].find_all('td')
            flag = False
            if(len(tds) > 3):
                try:
                    a = tds[2].find('a')['href']
                    urls.append("http://www.medlinks.ru/"+a)
                    flag = True
                    pageNum += 1
                                    
                except :
                    pass

            if flag:
                try:
                    getText(urls[-1])
                except:
                    errorNum +=1
                    print(str(pageNum), urls[-1])


        temp -= 50
    with open(folderName+"/"+str(num)+'.json', 'w+') as f:
        json.dump(d,f,ensure_ascii=False)
    num += 1

with open('all-urls.txt', 'w+') as f:
    for i in urls:
        f.write(i + '\n')
        
print(len(urls))
print(str(pageNum),str(errorNum))

http://www.medlinks.ru/modules.php?op=modload&name=Forum&file=viewforum&forum=106
-----
0
http://www.medlinks.ru/modules.php?op=modload&name=Forum&file=viewforum&forum=110
-----
4650
1143 http://www.medlinks.ru/modules.php?op=modload&name=Forum&file=viewtopic&topic=32737&forum=110
1400 http://www.medlinks.ru/modules.php?op=modload&name=Forum&file=viewtopic&topic=38767&forum=110
http://www.medlinks.ru/modules.php?op=modload&name=Forum&file=viewforum&forum=113
-----
250
http://www.medlinks.ru/modules.php?op=modload&name=Forum&file=viewforum&forum=115
-----
2650
http://www.medlinks.ru/modules.php?op=modload&name=Forum&file=viewforum&forum=117
-----
0
http://www.medlinks.ru/modules.php?op=modload&name=Forum&file=viewforum&forum=118
-----
350
http://www.medlinks.ru/modules.php?op=modload&name=Forum&file=viewforum&forum=120
-----
6000
9123 http://www.medlinks.ru/modules.php?op=modload&name=Forum&file=viewtopic&topic=35537&forum=120
http://www.medlinks.ru/modules.php?op=modload&name=Forum&fil

In [81]:
#d = {}

In [73]:
def getText(url):
    page =  s.get(url)
    soup = BeautifulSoup(page.content, 'lxml')
    mu = soup.find('body')
    mu = mu.find("div")
    mu = mu.find_all("table")
    mu = mu[30].find_all("tr")[1:]
    mu = mu[:-1]
    h1 = soup.find("h1").text
    d[h1] = []
    for i in mu:
        tds = i.find_all('td')
        if(len(tds) < 2):
            continue
            
        [j.decompose() for j in tds[1].find_all('b')]
#         [j.decompose() for j in tds[1].find_all('big')]
        [j.decompose() for j in tds[1].find_all('img')]
        [j.decompose() for j in tds[1].find_all('a')]

        t = str(tds[1].text)
#         print("bef-----------------------------------")
#         print(t)
        if t.find('Опубликовано')!=-1:
            t = t[:t.find("Опубликовано")] + t[t.find("Опубликовано") + len("Опубликовано: 07-11-2017 22:18"):]
            
        if t.find('[ Это сообщение было отред')!=-1:
            t = t[:t.find("[ Это сообщение было отред")] + t[t.find("]", t.find("[ Это сообщение было отред") + len("[ Это сообщение было отред") )+1 :]

        if t.find('&nbsp'):
            t = t[:t.find('&nbsp')]+ t[t.find("&nbsp")+5:]
        while t.find('&nbsp')!=-1:
            t = t[:t.find('&nbsp')]+ t[t.find("&nbsp")+5:]
        t = re.sub("[0-9]","", t)
#         print("aft-----------------------------------")
        d[h1].append(t)   
#         print(d[h1][-1])
        while d[h1][-1].find('\xa0')!=-1:
            d[h1][-1] =  d[h1][-1][ :d[h1][-1].find('\xa0')] + d[h1][-1][ d[h1][-1].find('\xa0')+ len("\xa0"):]
        while d[h1][-1].find('\r')!=-1:
            d[h1][-1] =  d[h1][-1][ :d[h1][-1].find('\r')] + d[h1][-1][ d[h1][-1].find('\r')+ len("\r"):]
            
#         print(t)
    
#     return d

         
# for i in urls[:1]:
#     print(i)
#     tt = getText(i)
#     print(tt) 
    
# print(len(d)) 
# for i in d:
#     print(len(d[i]))

In [75]:
# for i in d:
#     print(i)
#     print(d[i])
# print(len(d)) 
# for i in d:
#     print(len(d[i]))

print(len(links))

164
2
2
3
2
2
5
3
4
2
2
6
2
2
10
2
3
4
1
3
2
2
2
3
7
3
5
3
3
2
8
2
6
4
2
5
10
6
2
5
2
3
2
2
2
2
3
6
4
2
3
10
2
2
2
2
2
3
2
8
5
2
6
2
2
3
4
2
2
3
2
4
2
2
4
2
3
4
2
2
2
2
2
6
4
2
4
2
5
4
7
3
8
2
3
3
10
1
2
3
3
2
5
2
2
2
8
2
6
4
2
7
4
7
4
7
3
5
8
5
10
1
2
8
1
5
2
1
6
1
1
1
1
1
4
3
1
3
3
2
2
4
2
3
5
2
1
3
1
1
3
1
1
1
1
1
1
3
1
1
2
2
2
2
2


In [53]:
t = '''В данном случае заочно не ответишь, необходим осмотр и консультация. 

персональный сайт врача Огородникова Дениса Васильевича: http://reproductologist.com/

[ Это сообщение было отредактировано 24-03-2013 09:42 пользователем/модератором OgorodnikovDV ]'''

if t.find('[ Это сообщение было отред')!=-1:
    t = t[:t.find("[ Это сообщение было отред")] + t[t.find("]", t.find("[ Это сообщение было отред") + len("[ Это сообщение было отред") )+1 :]

print(t)







В данном случае заочно не ответишь, необходим осмотр и консультация. 

персональный сайт врача Огородникова Дениса Васильевича: http://reproductologist.com/


